In [1]:
import datetime

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.client import device_lib
import tensorflow as tf
import numpy as np
import os
import random
import time
from scipy.misc import imread, imresize
from imagenet_classes import class_names


train_dir="/home/cslee/ILSVRC2012/img_train"
batch_size = 32
total_epoch = 1    
learning_rate = 0.001
class_size = 1000
current_milli_time = lambda: int(round(time.time() * 1000))

In [2]:
def list_images(directory):
    """
    Get all the images and labels in directory/label/*.jpg
    """
    labels = os.listdir(directory)

    labels.sort()
    files_and_labels = []
    for label in labels:
        for f in os.listdir(os.path.join(directory, label)):
            files_and_labels.append((os.path.join(directory, label, f), label))

    filenames, labels = zip(*files_and_labels)
    filenames = list(filenames)
    labels = list(labels)
    
    unique_labels = []
    
    unique_labels.append(labels[0])
    compare_str = labels[0]
    for i in range(len(labels)):
        if labels[i] != compare_str :
            compare_str=labels[i]
            unique_labels.append(labels[i])

    label_to_int = {}
    for i, label in enumerate(unique_labels):
        label_to_int[label] = i
    
    labels = [label_to_int[l] for l in labels]
    
    training_data = []    
    for i in range(len(filenames)):
        training_data.append((filenames[i], labels[i])) #insert tuple to list
#     print(labels)
    return training_data    

In [3]:
def _parse_function(filename):
    image = imread(filename, mode = 'RGB')
    resized_image = imresize(image, (224, 224))
    resized_image = resized_image.astype(np.float32)
    mean = np.array([123.68, 116.779, 103.939])[np.newaxis, np.newaxis, :]
    resized_image = resized_image - mean
    return resized_image

In [4]:
# _parse_function('/home/cslee/ILSVRC2012/img_train/n01440764/n01440764_14506.JPEG')

In [5]:
def check_available_gpus():
    local_devices = device_lib.list_local_devices()
    gpu_names = [x.name for x in local_devices if x.device_type == 'GPU']
    gpu_num = len(gpu_names)

    print('{0} GPUs are detected : {1}'.format(gpu_num, gpu_names))

    return gpu_num


In [6]:
def model(X, reuse=False):
    print("input : ", X.shape)
    with tf.variable_scope('L1_1', reuse=reuse):
        L1 = tf.layers.conv2d(X, 64, [3, 3], activation = 'relu',reuse=reuse)
    with tf.variable_scope('L1_2', reuse=reuse):
        L1 = tf.layers.conv2d(L1, 64, [3, 3], activation = 'relu',reuse=reuse)
        L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
        #L1 = tf.layers.dropout(L1, 0.7, True)

    with tf.variable_scope('L2_1', reuse=reuse):
        L2 = tf.layers.conv2d(L1, 128, [3, 3],activation = 'relu',reuse=reuse)
    
    with tf.variable_scope('L2_2', reuse=reuse):
        L2 = tf.layers.conv2d(L2, 128, [3, 3],activation = 'relu',reuse=reuse)
        L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
        #L2 = tf.layers.dropout(L2, 0.7, True)

    with tf.variable_scope('L3_1', reuse=reuse):
        L3 = tf.layers.conv2d(L2, 256, [3, 3],activation = 'relu',reuse=reuse)
    
    with tf.variable_scope('L3_2', reuse=reuse):
        L3 = tf.layers.conv2d(L3, 256, [3, 3],activation = 'relu',reuse=reuse)
    with tf.variable_scope('L3_3', reuse=reuse):
        L3 = tf.layers.conv2d(L3, 256, [3, 3],activation = 'relu',reuse=reuse)
        L3 = tf.layers.max_pooling2d(L3, [2, 2], [2, 2])

    with tf.variable_scope('L4_1', reuse=reuse):
        L4 = tf.layers.conv2d(L3, 512, [3, 3],activation = 'relu',reuse=reuse)
    with tf.variable_scope('L4_2', reuse=reuse):
        L4 = tf.layers.conv2d(L4, 512, [3, 3],activation = 'relu',reuse=reuse)
    with tf.variable_scope('L4_3', reuse=reuse):
        L4 = tf.layers.conv2d(L4, 512, [3, 3],activation = 'relu',reuse=reuse)
        L4 = tf.layers.max_pooling2d(L4, [2, 2], [2, 2])

    with tf.variable_scope('L5_1', reuse=reuse):
        L5 = tf.layers.conv2d(L4, 512, [3, 3],activation = 'relu',reuse=reuse)
    with tf.variable_scope('L5_2', reuse=reuse):
        L5 = tf.layers.conv2d(L5, 512, [3, 3],activation = 'relu',reuse=reuse)
    with tf.variable_scope('L5_3', reuse=reuse):
        L5 = tf.layers.conv2d(L5, 512, [3, 3],activation = 'relu',reuse=reuse)
        L5 = tf.layers.max_pooling2d(L5, [2, 2], [2, 2])


    with tf.variable_scope('LF_1', reuse=reuse):
        LF = tf.layers.flatten(L5)
    with tf.variable_scope('LF_2', reuse=reuse):
        LF = tf.layers.dense(LF, 4096, activation= 'relu',reuse=reuse)
    with tf.variable_scope('LF_3', reuse=reuse):
        LF = tf.layers.dense(LF, 4096, activation= 'relu',reuse=reuse)
    with tf.variable_scope('LF_4', reuse=reuse):
        LF = tf.layers.dense(LF, class_size, activation=None,reuse=reuse)
    return LF


In [7]:
if __name__ == '__main__':
    # need to change learning rates and batch size by number of GPU
    training_dataset = list_images(train_dir)
    length = len(training_dataset)
    print(length)

    gpu_num = check_available_gpus()
    

    X = tf.placeholder(tf.float32, [None, 224, 224, 3])
    Y = tf.placeholder(tf.float32, [None, class_size])

    losses = []
    X_A = tf.split(X, int(gpu_num))
    Y_A = tf.split(Y, int(gpu_num))


1281167
2 GPUs are detected : ['/device:GPU:0', '/device:GPU:1']


In [8]:
    '''
    Multi GPUs Usage
    Results on P40
     * Single GPU computation time: 0:00:22.252533
     * 2 GPU computation time: 0:00:12.632623
     * 4 GPU computation time: 0:00:11.083071
     * 8 GPU computation time: 0:00:11.990167
     
    Need to change batch size and learning rates
         for training more efficiently
    
    Reference: https://research.fb.com/wp-content/uploads/2017/06/imagenet1kin1h5.pdf
    '''
    for gpu_id in range(int(gpu_num)):
        with tf.device(tf.DeviceSpec(device_type="GPU", device_index=gpu_id)):
            with tf.variable_scope(tf.get_variable_scope(), reuse=(gpu_id > 0)):
                cost = tf.nn.softmax_cross_entropy_with_logits_v2(
                                logits=model(X_A[gpu_id], gpu_id > 0),
                                labels=Y_A[gpu_id])
                losses.append(cost)

    loss = tf.reduce_mean(tf.concat(losses, axis=0))

input :  (?, 224, 224, 3)
input :  (?, 224, 224, 3)


In [ ]:
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(
        loss, colocate_gradients_with_ops=True)  # Important!

    init = tf.global_variables_initializer()
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
    sess.run(init)

#     mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)
#     total_batch = int(mnist.train.num_examples/batch_size)
#     print("total: %s, %s, %s" % (mnist.train.num_examples, total_batch, batch_size))

    start_time = datetime.datetime.now()

    for epoch in range(total_epoch):
        total_cost = 0
        p = np.random.permutation(length)
        iter_image = np.array([np.array(image) for image, _ in training_dataset])    
        iter_label = np.array([int(label) for _, label in training_dataset])
        
        iter_image, iter_label = iter_image[p], iter_label[p]
        print("Epoch {}\n".format(epoch))
        
        for n_idx in range(0, length, batch_size):#one batch
            up_to_idx = min(n_idx + batch_size, length)
            iter_image_batch = iter_image[n_idx:up_to_idx]
            iter_label_batch =  iter_label[n_idx:up_to_idx]
            #get resized image
            image_list=[]
            label_list=[]
            for i in range(len(iter_image_batch)):
                image_path = iter_image_batch[i]
                label_list.append(iter_label_batch[i])
                resized_image = _parse_function(image_path)
                image_list.append(resized_image)

            one_hot_label = np.eye(class_size)[label_list]
            np.array(one_hot_label)
#             print(one_hot_label.shape)
#             print(image_list)
            _, cost_val = sess.run([optimizer, loss],
                                   feed_dict={X: image_list,
                                              Y: one_hot_label})
            total_cost += cost_val
            print("batch cost : %s" % cost_val)

        print("total cost : %s" % total_cost)

    print("--- Training time : {0} seconds /w {1} GPUs ---".format(
        datetime.datetime.now() - start_time, gpu_num))

Epoch 0



/home/yongwoo/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/home/yongwoo/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


batch cost : 6.9222097
batch cost : 316.27603
batch cost : 6.913548
batch cost : 6.908338
batch cost : 6.8932457
batch cost : 8.893727
batch cost : 6.921254
batch cost : 6.9034605
batch cost : 6.906917
batch cost : 6.9067035
batch cost : 6.9055285
batch cost : 6.911097
batch cost : 6.913173
batch cost : 6.910367
batch cost : 6.9349513
batch cost : 6.9135027
batch cost : 6.917733
batch cost : 6.914538
batch cost : 6.898601
batch cost : 6.9059057
batch cost : 6.894648
batch cost : 6.8823442
batch cost : 7.1177263
batch cost : 6.9328976
batch cost : 6.908776
batch cost : 6.910451
batch cost : 6.907295
batch cost : 6.9105086
batch cost : 6.9143825
batch cost : 6.912183
batch cost : 6.8967276
batch cost : 6.8970394
batch cost : 6.916153
batch cost : 6.9113026
batch cost : 6.9116163
batch cost : 6.900481
batch cost : 6.9272704
batch cost : 6.897947
batch cost : 6.914871
batch cost : 6.9089413
batch cost : 6.9133353
batch cost : 6.9096136
batch cost : 6.8998547
batch cost : 6.8937197
batch co

batch cost : 6.9062448
batch cost : 6.9037495
batch cost : 6.9104137
batch cost : 6.914645
batch cost : 6.905674
batch cost : 6.9028645
batch cost : 6.907408
batch cost : 6.9101925
batch cost : 6.9103546
batch cost : 6.9138813
batch cost : 6.910305
batch cost : 6.913285
batch cost : 6.9090433
batch cost : 6.905301
batch cost : 6.904079
batch cost : 6.911126
batch cost : 6.912338
batch cost : 6.9146795
batch cost : 6.909294
batch cost : 6.9100227
batch cost : 6.912184
batch cost : 6.90703
batch cost : 6.906373
batch cost : 6.906783
batch cost : 6.910759
batch cost : 6.9129252
batch cost : 6.91618
batch cost : 6.895376
batch cost : 6.912312
batch cost : 6.9064817
batch cost : 6.9209304
batch cost : 6.9101257
batch cost : 6.907505
batch cost : 6.9077053
batch cost : 6.9122686
batch cost : 6.8947315
batch cost : 6.9161496
batch cost : 6.901408
batch cost : 6.92009
batch cost : 6.904353
batch cost : 6.897667
batch cost : 6.910072
batch cost : 6.9030476
batch cost : 6.905739
batch cost : 6.9

batch cost : 6.917271
batch cost : 6.899528
batch cost : 6.9003677
batch cost : 6.8898106
batch cost : 6.911709
batch cost : 6.90239
batch cost : 6.913107
batch cost : 6.908416
batch cost : 6.898355
batch cost : 6.8998766
batch cost : 6.9066477
batch cost : 6.8948345
batch cost : 6.91146
batch cost : 6.9175663
batch cost : 6.906352
batch cost : 6.903066
batch cost : 6.9040427
batch cost : 6.9114304
batch cost : 6.919553
batch cost : 6.8907566
batch cost : 6.901432
batch cost : 6.9168234
batch cost : 6.894717
batch cost : 6.9003496
batch cost : 6.9032183
batch cost : 6.9106803
batch cost : 6.904723
batch cost : 6.904377
batch cost : 6.9105716
batch cost : 6.908271
batch cost : 6.9045506
batch cost : 6.898917
batch cost : 6.902306
batch cost : 6.9085965
batch cost : 6.896524
batch cost : 6.9054613
batch cost : 6.9093814
batch cost : 6.894869
batch cost : 6.9133644
batch cost : 6.9099684
batch cost : 6.914493
batch cost : 6.9019966
batch cost : 6.908826
batch cost : 6.9160647
batch cost :

batch cost : 6.896433
batch cost : 6.910989
batch cost : 6.924063
batch cost : 6.9069567
batch cost : 6.905777
batch cost : 6.90619
batch cost : 6.912488
batch cost : 6.911699
batch cost : 6.9107647
batch cost : 6.9199543
batch cost : 6.9169207
batch cost : 6.8993444
batch cost : 6.9057417
batch cost : 6.903288
batch cost : 6.905759
batch cost : 6.919733
batch cost : 6.911765
batch cost : 6.902005
batch cost : 6.915379
batch cost : 6.9011965
batch cost : 6.9128966
batch cost : 6.918127
batch cost : 6.9048243
batch cost : 6.909219
batch cost : 6.9101496
batch cost : 6.912147
batch cost : 6.9153852
batch cost : 6.8949947
batch cost : 6.8935976
batch cost : 6.9072075
batch cost : 6.9091806
batch cost : 6.919236
batch cost : 6.9064226
batch cost : 6.9119196
batch cost : 6.912445
batch cost : 6.916682
batch cost : 6.9181356
batch cost : 6.9135585
batch cost : 6.896225
batch cost : 6.91096
batch cost : 6.9118414
batch cost : 6.918144
batch cost : 6.908282
batch cost : 6.8929987
batch cost : 